# Summarize by Ward
last updated 3/3/24

<a name="read"></a>
# 1. Read and Prepare Geocoded Fines and 311 Complaints Data

In [1]:
import pandas as pd

In [2]:
df_fines = pd.read_csv("../../data/05-finalized/dockets-summary.csv")
df_fines.head()

,docket,dept,violation_address,lat,long,community,ward_1523,violation_date,n_records
0,19DS68300L,STRTSAN,4710 S WESTERN AVE,41.807859,-87.68479703503766,BRIGHTON PARK,15,2019-11-13,2
1,19DS69216L,STRTSAN,1425 W MORSE AVE,42.0074513,-87.6668285,ROGERS PARK,49,2019-11-13,1
2,19DS70010L,STRTSAN,715 E 47TH ST,41.8093383,-87.6080127,GRAND BOULEVARD,4,2019-11-13,1
3,19DS72153L,STRTSAN,300 W WASHINGTON ST,41.8818694,-87.7401431,WEST GARFIELD PARK,28,2019-11-12,5
4,19DS72160L,STRTSAN,6929 N SHERIDAN RD,41.9598134,-87.654693,UPTOWN,46,2019-11-14,1


In [3]:
df_311 = pd.read_csv("../../data/05-finalized/311-complaints-unshoveled.csv")
df_311.head()

,SR_NUMBER,SR_SHORT_CODE,CREATED_DATE,STREET_ADDRESS,COMMUNITY_AREA,WARD,STATUS,ORIGIN,CLOSED_DATE,LATITUDE,LONGITUDE,SR_TYPE,year,month,date,season,GEOID,COMMUNITY_NAME
0,SR20-05649092,SWSNOREM,2020-12-17 13:24:37,5200 S BLACKSTONE AVE,41,4,Completed,Phone Call,2020-12-18 15:12:51,41.800909,-87.590515,Snow – Uncleared Sidewalk Complaint,2020,12,2020-12-17,2020-2021,41,Hyde Park
1,SR21-00001364,SWSNOREM,2021-01-01 11:40:33,66 E CHESTNUT ST,8,42,Completed,Mobile Device,2021-01-04 14:33:44,41.898399,-87.625723,Snow – Uncleared Sidewalk Complaint,2021,1,2021-01-01,2020-2021,8,Near North Side
2,SR21-00000774,SWSNOREM,2021-01-01 09:11:15,2648 N WHIPPLE ST,22,32,Completed,Internet,2021-01-06 13:37:58,41.930233,-87.703706,Snow – Uncleared Sidewalk Complaint,2021,1,2021-01-01,2020-2021,22,Logan Square
3,SR20-05723378,SWSNOREM,2020-12-31 07:19:37,5015 N SPRINGFIELD AVE,14,39,Completed,Mobile Device,2021-01-08 15:30:57,41.972439,-87.725430,Snow – Uncleared Sidewalk Complaint,2020,12,2020-12-31,2020-2021,14,Albany Park
4,SR21-00014080,SWSNOREM,2021-01-04 14:07:09,3757 N BELL AVE,5,47,Completed,Internet,2021-01-06 11:37:30,41.950316,-87.684472,Snow – Uncleared Sidewalk Complaint,2021,1,2021-01-04,2020-2021,5,North Center


In [4]:
len(df_fines)

1918

In [5]:
len(df_311)

21079

<a name="community"></a>
# Aggregate Dockets by Ward

In [6]:
# index_to_drop = df_wards[df_wards['ward'] == 'unknown'].index
# df_wards = df_wards.drop(index=index_to_drop)
#df_ward_fines= df_fines.drop(index=(df_wards[df_wards['ward']=='unknown'].index))
df_ward_fines = df_fines.pivot_table(index='ward_1523',
                             columns='dept',
                             values='docket',
                             aggfunc=['count'],
                             fill_value=0).reset_index()
# Flatten the MultiIndex in columns
df_ward_fines.columns = ['_'.join(col).strip() for col in df_ward_fines.columns.values]

df_ward_fines = df_ward_fines.rename(columns={'ward_1523_':'ward'})
df_ward_fines.rename(columns=lambda x: x.replace('count_','n_'), inplace=True)

# Reset the index to flatten it
df_ward_fines.reset_index(drop=True, inplace=True)

df_ward_fines['n_dockets'] = df_ward_fines.filter(like='n_').sum(axis=1)
df_ward_fines['ward']=df_ward_fines['ward'].astype(str)

df_ward_fines.head()

,ward,n_BAFCONP,n_POLICE,n_STRTSAN,n_TRANPORT,n_unknown,n_dockets
0,1,0,0,1,61,0,62
1,10,0,0,0,9,0,9
2,11,0,0,3,80,0,83
3,12,0,0,3,22,0,25
4,13,0,1,2,26,0,29


In [7]:
# confirm totals
df_ward_fines['n_dockets'].sum()

1918

In [8]:
len(df_ward_fines)

51

# Aggregate 311 Complaints by Ward

In [9]:
df_ward_311 = df_311.groupby('WARD').size().reset_index(name='n_311_complaints')
df_ward_311.rename(columns={'WARD':'ward'}, inplace = True)
df_ward_311['ward'].astype(int).astype(str)
df_ward_311.head()

,ward,n_311_complaints
0,1,1215
1,2,770
2,3,386
3,4,478
4,5,265


In [10]:
df_ward_311['n_311_complaints'].sum()
df_ward_311['ward'] = df_ward_311['ward'].astype(int).astype(str)

In [11]:
len(df_ward_311)

50

# Merge Datasets

### Merge Dockets and 311 complaints

In [12]:
df_ward = pd.merge(df_ward_fines,df_ward_311, on = 'ward')
df_ward.head()

,ward,n_BAFCONP,n_POLICE,n_STRTSAN,n_TRANPORT,n_unknown,n_dockets,n_311_complaints
0,1,0,0,1,61,0,62,1215
1,10,0,0,0,9,0,9,126
2,11,0,0,3,80,0,83,369
3,12,0,0,3,22,0,25,207
4,13,0,1,2,26,0,29,139


In [13]:
# confirm I have 50 wards
len(df_ward)

50

# Calculations

In [14]:
df_ward['complaint_ratio'] = df_ward['n_TRANPORT']/df_ward['n_311_complaints']
df_ward.head()

,ward,n_BAFCONP,n_POLICE,n_STRTSAN,n_TRANPORT,n_unknown,n_dockets,n_311_complaints,complaint_ratio
0,1,0,0,1,61,0,62,1215,0.050206
1,10,0,0,0,9,0,9,126,0.071429
2,11,0,0,3,80,0,83,369,0.216802
3,12,0,0,3,22,0,25,207,0.106280
4,13,0,1,2,26,0,29,139,0.187050


### confirm totals

In [15]:
df_ward.sum()

ward                1101112131415161718192202122232425262728293303...
n_BAFCONP                                                           2
n_POLICE                                                           25
n_STRTSAN                                                         497
n_TRANPORT                                                       1388
n_unknown                                                           1
n_dockets                                                        1913
n_311_complaints                                                21079
complaint_ratio                                              4.067015
dtype: object

# Export

In [16]:
df_ward.to_csv("../../results/ssw02-fines/ward-summary.csv", index= False)